In [15]:
import numpy as np
import math
import matplotlib.pyplot as plt
from akorn import *
from flh import *
%load_ext autoreload
%autoreload 2

%matplotlib qt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
%load_ext autoreload
%autoreload 2

import rpy2
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr, data



utils = importr('utils')
base = importr('base')
glmgen = importr('glmgen')
%matplotlib qt

std = 0.3
var = std**2

# mse_akorn = []
# mse_tf = []



def doppler(x, epsilon):
    return np.sin(2 * np.pi * (1 + epsilon) / (x + epsilon))

from util import piecewise_linear

print("Done!")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Done!


In [35]:
mse_akorn = []
mse_addle = []
mse_tf = []
mse_tf_dof = []

Ts = [50, 100, 200, 500, 1000, 2500, 5000]

runs_per_T = 30

from util import *

akorn_fv = None
dof_fv = None
tf_fv = None
addle_fv = None
    
for T in Ts:
    print("T = ", T)
    X = np.linspace(0, T, T)/T 
    Xr = robjects.FloatVector(X)

    k = 5
    G = np.array([0]*(T-k) + [1 + j for j in range(k)])

    mses_T_tf = []
    mses_T_tf_dof = []
    mses_T_akorn = []
    mses_T_addle = []

    for l in range(runs_per_T):
        print("Run number " + str(l))
        Y = (G + np.random.normal(0, std, T))

        ak = AKORN(X, Y, var)
        ak.train()
        fv = ak.preds
        
        online = FLH(X, Y, T, lr = 1/8*(5**2))
        online.run()
        fv_addle = online.predictions
        
        mses_T_addle.append(((fv_addle - G)**2).mean())


        mses_T_akorn.append(((fv - G)**2).mean())


        Yr = robjects.FloatVector(Y)
        tf = glmgen.trendfilter(Xr, Yr, k = 1)
        rcode1 = "predict(%s)" %(tf.r_repr())
        tf_fit = np.array(robjects.r(rcode1)).T

        best_mse = float('inf')
        best_mse_est = float('inf')
        best_mse_dof = float('inf')
        best_tf_fit = None
        best_tf_fit_dof = None
        for t in tf_fit:
            tf_mse = ((t - G)**2).mean() #Oracle risk
            tf_mse_est = (np.sum(((t - Y)**2)) + 2*var*count_linear_pieces(t)) #Stein risk estimate
            if tf_mse < best_mse:
                best_mse = tf_mse
                best_tf_fit = t
            if tf_mse_est < best_mse_est:
                best_mse_est = tf_mse_est
                best_mse_dof = tf_mse
                best_tf_fit_dof = t
                
        mses_T_tf.append(((best_tf_fit - G)**2).mean())
        mses_T_tf_dof.append(((best_tf_fit_dof - G)**2).mean())
        
        akorn_fv = fv
        addle_fv = fv_addle
        tf_dof_fv = best_tf_fit_dof
        tf_fv = best_tf_fit

    #     Z_tf.append(best_tf_fit)
    mse_tf.append(mses_T_tf)
    mse_tf_dof.append(mses_T_tf_dof)
    mse_addle.append(mses_T_addle)
    mse_akorn.append(mses_T_akorn)
    

avg_mses_akorn = [np.array(mse_T).mean() for mse_T in mse_akorn]
avg_mses_addle = [np.array(mse_T).mean() for mse_T in mse_addle]
avg_mses_tf = [np.array(mse_T).mean() for mse_T in mse_tf]
avg_mses_tf_dof = [np.array(mse_T).mean() for mse_T in mse_tf_dof]
print("Done!")

T =  50
Run number 0
lr =  0.005710486779719426
run
0
Run number 1
lr =  0.004684630258408144
run
0
Run number 2
lr =  0.00474874799269433
run
0
Run number 3
lr =  0.00452081201784348
run
0
Run number 4
lr =  0.004074568563292229
run
0
Run number 5
lr =  0.0053880897705603946
run
0
Run number 6
lr =  0.005042013826019119
run
0
Run number 7
lr =  0.004106612779983706
run
0
Run number 8
lr =  0.004986257001715874
run
0
Run number 9
lr =  0.005001873614296806
run
0
Run number 10
lr =  0.004633454677910428
run
0
Run number 11
lr =  0.004907882582340915
run
0
Run number 12
lr =  0.005512366998746664
run
0
Run number 13
lr =  0.0063033653118781995
run
0
Run number 14
lr =  0.005287363074221106
run
0
Run number 15
lr =  0.0051663586303311824
run
0
Run number 16
lr =  0.005849753956996293
run
0
Run number 17
lr =  0.004425843169759282
run
0
Run number 18
lr =  0.005017970382969053
run
0
Run number 19
lr =  0.004965429375914152
run
0
Run number 20
lr =  0.004877278648654017
run
0
Run number 21


0
1000
2000
Run number 20
lr =  0.0056025250041649224
run
0
1000
2000
Run number 21
lr =  0.0046244697443759706
run
0
1000
2000
Run number 22
lr =  0.004128362720845677
run
0
1000
2000
Run number 23
lr =  0.004265710164879238
run
0
1000
2000
Run number 24
lr =  0.004457784305502821
run
0
1000
2000
Run number 25
lr =  0.005441749984890737
run
0
1000
2000
Run number 26
lr =  0.005463897904145244
run
0
1000
2000
Run number 27
lr =  0.005488036016075211
run
0
1000
2000
Run number 28
lr =  0.0045110057653393
run
0
1000
2000
Run number 29
lr =  0.00636491853478624
run
0
1000
2000
T =  5000
Run number 0
lr =  0.00487406904346111
run
0
1000
2000
3000
4000
Run number 1
lr =  0.005783477998953197
run
0
1000
2000
3000
4000
Run number 2
lr =  0.004985757337629793
run
0
1000
2000
3000
4000
Run number 3
lr =  0.00516716145830515
run
0
1000
2000
3000
4000
Run number 4
lr =  0.004880937149257523
run
0
1000
2000
3000
4000
Run number 5
lr =  0.005806654954427606
run
0
1000
2000
3000
4000
Run number 6
lr

In [ ]:

mse_tf = []
mse_tf_dof = []
# Ts = [100, 5000, 100_000, 200_000, 500_000]
Ts = [50, 100, 200, 500, 1000, 2500, 5000]
# Ts = [50, 100]

runs_per_T = 20

from util import *


    
for T in Ts:
    print("T = ", T)
    X = np.linspace(0, T, T)/T 
    Xr = robjects.FloatVector(X)

    k = 5
    G = np.array([0]*(T-k) + [1 + j for j in range(k)])

    mses_T_tf = []
    mses_T_tf_dof = []

    for l in range(runs_per_T):
        print("Run number " + str(l))
        Y = (G + np.random.normal(0, std, T))

        Yr = robjects.FloatVector(Y)
        tf = glmgen.trendfilter(Xr, Yr, k = 1)
        rcode1 = "predict(%s)" %(tf.r_repr())
        tf_fit = np.array(robjects.r(rcode1)).T

        best_mse = float('inf')
        best_mse_est = float('inf')
        best_mse_dof = float('inf')
        best_tf_fit = None
        best_tf_fit_dof = None
        for t in tf_fit:
            tf_mse = ((t - G)**2).mean() #Oracle risk
            tf_mse_est = (np.sum(((t - Y)**2)) + 2*var*count_linear_pieces(t)) #Stein risk estimate
            if tf_mse < best_mse:
                best_mse = tf_mse
                best_tf_fit = t
            if tf_mse_est < best_mse_est:
                best_mse_est = tf_mse_est
                best_mse_dof = tf_mse
                best_tf_fit_dof = t
                
        mses_T_tf.append(((best_tf_fit - G)**2).mean())
        mses_T_tf_dof.append(((best_tf_fit_dof - G)**2).mean())
        

        tf_dof_fv = best_tf_fit_dof
        tf_fv = best_tf_fit

    #     Z_tf.append(best_tf_fit)
    mse_tf.append(mses_T_tf)
    mse_tf_dof.append(mses_T_tf_dof)

avg_mses_tf = [np.array(mse_T).mean() for mse_T in mse_tf]
avg_mses_tf_dof = [np.array(mse_T).mean() for mse_T in mse_tf_dof]
print("Done!")

In [97]:
from sklearn.linear_model import LinearRegression
akorn_ref = LinearRegression().fit(np.log(np.array(Ts)).reshape(-1, 1), np.log(avg_mses_akorn))
addle_ref = LinearRegression().fit(np.log(np.array(Ts)).reshape(-1, 1), np.log(avg_mses_addle))
tf_ref = LinearRegression().fit(np.log(np.array(Ts)).reshape(-1, 1), np.log(np.array(avg_mses_tf)))
tf_dof_ref = LinearRegression().fit(np.log(np.array(Ts)).reshape(-1, 1), np.log(np.array(avg_mses_tf_dof)))

In [108]:
import pywt
def wavelet_denoise(Y, a, std):
        coeffs = pywt.wavedec(Y, "db2")
        thresholded = []
        for c in coeffs:
            thresholded.append(pywt.threshold(c, a*std*math.sqrt(2*math.log(T)), mode = "soft"))
        signal = pywt.waverec(thresholded, "db2")
        return signal
    
def oracle_wavelets(Y,G, std):
    best_fit = None
    best_mse = float('inf')
    best_a = None
    for a in np.linspace(1, 100, 100):
        sig = wavelet_denoise(Y, a, std)
        sig_mse = np.mean(np.square(sig - G))
        if sig_mse < best_mse:
            print("a = ", a)
            best_fit = sig
            best_mse = sig_mse
            best_a = a
    return (best_fit, best_mse, best_a)
mse_wav = []
wav_fv = None
runs_per_T = 20

for T in Ts:
    print("T = ", T)
    X = np.linspace(0, T, T)/T 

    k = 5
    G = np.array([0]*(T-k) + [1 + j for j in range(k)])

    mses_T_wav = []
    for l in range(runs_per_T):
        print("Run number " + str(l))
        Y = (G + np.random.normal(0, std, T))
        
        fv_wav = oracle_wavelets(Y, G, std)[0]
        mses_T_wav.append(((fv_wav - G)**2).mean())
        
        wav_fv =  fv_wav

    mse_wav.append(mses_T_wav)
    

avg_mses_wav = [np.array(mse_T).mean() for mse_T in mse_wav]
wav_ref = LinearRegression().fit(np.log(np.array(Ts)).reshape(-1, 1), np.log(np.array(avg_mses_wav)))
print("Done!")

T =  50
Run number 0
a =  1.0
Run number 1
a =  1.0
Run number 2
a =  1.0
Run number 3
a =  1.0
Run number 4
a =  1.0
Run number 5
a =  1.0
Run number 6
a =  1.0
Run number 7
a =  1.0
Run number 8
a =  1.0
Run number 9
a =  1.0
Run number 10
a =  1.0
Run number 11
a =  1.0
Run number 12
a =  1.0
Run number 13
a =  1.0
Run number 14
a =  1.0
Run number 15
a =  1.0
Run number 16
a =  1.0
Run number 17
a =  1.0
Run number 18
a =  1.0
Run number 19
a =  1.0
T =  100
Run number 0
a =  1.0
Run number 1
a =  1.0
Run number 2
a =  1.0
Run number 3
a =  1.0
Run number 4
a =  1.0
Run number 5
a =  1.0
Run number 6
a =  1.0
Run number 7
a =  1.0
Run number 8
a =  1.0
Run number 9
a =  1.0
Run number 10
a =  1.0
Run number 11
a =  1.0
Run number 12
a =  1.0
Run number 13
a =  1.0
Run number 14
a =  1.0
Run number 15
a =  1.0
Run number 16
a =  1.0
Run number 17
a =  1.0
Run number 18
a =  1.0
Run number 19
a =  1.0
T =  200
Run number 0
a =  1.0
Run number 1
a =  1.0
Run number 2
a =  1.0
Run numb

In [115]:
def quick_round_str(z):
    return str(round(z, 5))

plt.clf()
from util import subplots_centered
T = Ts[-1]
X = np.linspace(0, T, T)/T 

k = 5
G = np.array([0]*(T-k) + [1 + j for j in range(k)])

f, ax = plt.subplots(1, 2)
# f, ax = subplots_
ax0 = ax[0]
ax1 = ax[1]
ax1.plot(range(T), G, color = "black")
ax1.plot(range(T), tf_fv, color = "tab:blue")
ax1.plot(range(T), Y, alpha = 0.15, color = "black")
ax1.plot(range(T), akorn_fv, color = "tab:orange")

ax1.yaxis.set_visible(False)

ax0.set_xlabel("n")
ax0.set_ylabel("MSE")

# ax0.set_ylim([10**(-4), 10**(-1)])
# ax1.set_ylim([10**(-4), 10**(0)])

ax0.grid(True)


ax0.loglog(Ts, [T**(-2/5) for T in Ts], linestyle = "dotted", label = "$n^{-2/5}$ line", color = "black")
ax0.loglog(Ts, avg_mses_tf, marker = 'o', label = "oracle trend filtering", color = "tab:blue")
ax0.loglog(Ts, avg_mses_wav, marker = 'o', label = "wavelets", color = "purple")
ax0.loglog(Ts, avg_mses_tf_dof, marker = 'o', label = "tf dof", color = "blue")
ax0.loglog(Ts, avg_mses_akorn, marker = 'o', label = "akorn", color = "tab:orange")

f.legend(framealpha = 1)

In [98]:
print(tf_ref.coef_[0])
print(tf_dof_ref.coef_[0])
print(akorn_ref.coef_[0])

-0.3670865850257204
-0.38820283557060464
-0.6929287811105723
